<a href="https://colab.research.google.com/github/aniketsaha2310/MoodTune/blob/main/imagenetVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import PIL
import PIL.Image
import tensorflow as tf
import warnings

In [4]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
warnings.filterwarnings('ignore')

In [5]:
# Define data paths (using your provided paths)
train_path = "/content/drive/MyDrive/MoodTune-main/Dataset/train"
test_path = "/content/drive/MyDrive/MoodTune-main/Dataset/test"


In [6]:
# Image dimensions (adjust if necessary)
img_width, img_height = 224, 224

In [7]:
# Data preprocessing using ImageDataGenerator (optional)
train_datagen = ImageDataGenerator(rescale=1./255,  # Rescale pixel values to 0-1
                                   shear_range=0.2,  # Optional: random shear augmentation
                                   zoom_range=0.2,   # Optional: random zoom augmentation
                                   horizontal_flip=True)  # Optional: random horizontal flip

test_datagen = ImageDataGenerator(rescale=1./255)  # Rescale only for testing

In [8]:
# Create data generators (assuming data is organized in subdirectories for classes)
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'  # Assuming categorical labels
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical'  # Assuming categorical labels
)


Found 41275 images belonging to 4 classes.
Found 5140 images belonging to 4 classes.


In [9]:
# Load pre-trained VGG16 model with pre-trained weights (ImageNet)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))


58889256/58889256 [==============================] - 0s 0us/step


In [10]:
# Build the model
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, activation='softmax'))  # 4 classes for angry, happy, neutral, sad

In [12]:
#Summarize the Model
print(model.summary())  # Print a summary of the model architecture


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 21138500 (80.64 MB)
Trainable params: 21138500 (80.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),  # Adjust learning rate as needed
              loss=CategoricalCrossentropy(),
              metrics=['accuracy'])

In [15]:
# Train the model using the data generators
model.fit(train_generator,
          epochs= 7,  # Adjust epochs as needed
          validation_data=test_generator)


Epoch 1/7
  91/1290 [=>............................] - ETA: 23:00:03 - loss: 1.9780 - accuracy: 0.3183

KeyboardInterrupt: 

In [ ]:
# Save the model (optional)
model.save('vgg16_model.h5')